In [779]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
import scipy.stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
import urllib
import requests
import zipfile
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
from sklearn.decomposition import PCA
import geopandas as gpd
from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


# 1. data preparation

In [780]:
def get_LL84(url):
    data = pd.read_excel(url)
    cols = [x.encode('utf8').replace('\xc2\xb2', '2') for x in data.columns]
    data.columns = cols
    data.rename(columns = {'NYC Borough, Block, and Lot (BBL)':'BBL',
                           'NYC Borough, Block and Lot (BBL)':'BBL','Zip Code':'Zip',
                           'Site EUI (kBtu/ft2)':'Site EUI',
        'Site EUI\n(kBtu/ft2)':'Site EUI','DOF Benchmarking Submission Status':'Benchmarking Submission',
                           'Weather Normalized Source EUI\n(kBtu/ft2)':'Weather Normalized Source EUI',
                           'Weather Normalized Source EUI (kBtu/ft2)':'Weather Normalized Source EUI',
                           'Municipally Supplied Potable Water - Indoor Intensity (gal/ft2)':'Indoor Water Intensity(gal/ft2)',
                           'Indoor Water Intensity (All Water Sources)\n(gal/ft2)':'Indoor Water Intensity(gal/ft2)',
                           'Water per Square Foot':'Indoor Water Intensity(gal/ft2)',
                           'Total GHG Emissions\n(MtCO2e)':'GHG',
                           'Total GHG Emissions (Metric Tons CO2e)':'GHG',
                           'Municipally Supplied Potable Water - Indoor Intensity (gal/ft2)':'Indoor Water Intensity(gal/ft2)',
                               'Property Floor Area (Buildngs and Parking)\n(ft2)':'Property Floor Area(ft2)',
                           'DOF Property Floor Area (ft2)':'Property Floor Area(ft2)',
                               'DOF Property Floor Area (Buildngs and Parking)\n(ft2)':'Property Floor Area(ft2)',
                           'DOF Property Floor Area (ft²)':'Property Floor Area(ft2)',
                               'DOF Number of Buildings':'Number of Buildings',
                           'Number of Buildings - Self-reported':'Number of Buildings',
                          'Primary Property Type - Self Selected':'Reported Facility Type'},inplace=True)
    
    
    data = data[data['ENERGY STAR Score']>0][['BBL','Zip','Benchmarking Submission','Site EUI','Weather Normalized Source EUI',
                 'Indoor Water Intensity(gal/ft2)','Reported Water Method','ENERGY STAR Score'
                ,'GHG','Property Floor Area(ft2)','Reported Facility Type','Number of Buildings']]
    return data

In [781]:
#LL13 = get_LL84('http://www.nyc.gov/html/gbee/downloads/excel/2013_nyc_ll84_disclosure.xlsx')

In [782]:
#LL14 = get_LL84('http://www.nyc.gov/html/gbee/downloads/excel/150428_2014_nyc_ll84_disclosure.xlsx')

In [783]:
#LL15 = get_LL84('http://www.nyc.gov/html/gbee/downloads/excel/2015_nyc_cy2014__ll84_disclosure_data.xlsx')

In [784]:
LL16 = get_LL84('http://www.nyc.gov/html/gbee/downloads/excel/nyc_benchmarking_disclosure_data_reported_in_2016.xlsx')

In [785]:
PRICE = pd.read_csv('Housing_price/2016.csv')

## As we have only 4 years of data here, we would now only use data of 2016 

In [786]:
LL16.BBL = LL16.BBL.astype('str').str[:-2]

In [787]:
merged16 = pd.merge(LL16,PRICE,on='BBL')

In [788]:
merged16 = merged16[merged16.VALUE>0]

In [789]:
merged16.dropna(axis=0,inplace = True)

In [790]:
merged16.columns = [x.replace(' ','_') for x in merged16.columns]

In [791]:
merged16.VALUE = merged16.VALUE/merged16['Property_Floor_Area(ft2)']

In [792]:
merged16.head()

,BBL,Zip,Benchmarking_Submission,Site_EUI,Weather_Normalized_Source_EUI,Indoor_Water_Intensity(gal/ft2),Reported_Water_Method,ENERGY_STAR_Score,GHG,Property_Floor_Area(ft2),Reported_Facility_Type,Number_of_Buildings,VALUE
2,1014270028,10021.0,In Compliance,44.9,105.2,71.51,Manual,80.0,538.4,166432.0,Multifamily Housing,1.0,203.873053
3,1015180024,10128.0,In Compliance,91.3,180.7,64.86,ABS,10.0,699.3,114939.0,Multifamily Housing,1.0,211.138082
49,1000260021,10005.0,In Compliance,63.5,128.4,34.22,ABS,39.0,2163.1,493187.0,Multifamily Housing,1.0,231.713326
87,1000520021,10006.0,In Compliance,84.6,131.1,23.11,ABS,24.0,379.0,57945.0,Multifamily Housing,1.0,105.410303
108,1000680028,10038.0,In Compliance,107.7,231.7,88.05,ABS,40.0,534.2,71539.0,Hotel,1.0,421.168873


# PLUTO

In [793]:
urllib.urlretrieve('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_16v2%20.zip','pluto.zip')

('pluto.zip', <httplib.HTTPMessage instance at 0x0000000017A126C8>)

In [794]:
pluto = zipfile.ZipFile('pluto.zip')
pluto.extractall()

In [795]:
pluto = pd.concat(pd.read_csv('BORO_zip_files_csv/{}.csv'.format(x))for x in ['BK','BX','MN','QN','SI'])

C:\Users\sddz0\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (18,19,21,22,23,24,25,63,64,79) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\sddz0\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (18,19,21,22,63,64,79) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\sddz0\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (19,22,23,25,63,79) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\sddz0\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (18,19,21,22,63,64,76) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\sddz0\Anaconda2\lib\site-packages\ipy

In [796]:
pluto.columns

Index([u'Borough', u'Block', u'Lot', u'CD', u'CT2010', u'CB2010',
       u'SchoolDist', u'Council', u'ZipCode', u'FireComp', u'PolicePrct',
       u'HealthArea', u'SanitBoro', u'SanitDistrict', u'SanitSub', u'Address',
       u'ZoneDist1', u'ZoneDist2', u'ZoneDist3', u'ZoneDist4', u'Overlay1',
       u'Overlay2', u'SPDist1', u'SPDist2', u'SPDist3', u'LtdHeight',
       u'SplitZone', u'BldgClass', u'LandUse', u'Easements', u'OwnerType',
       u'OwnerName', u'LotArea', u'BldgArea', u'ComArea', u'ResArea',
       u'OfficeArea', u'RetailArea', u'GarageArea', u'StrgeArea',
       u'FactryArea', u'OtherArea', u'AreaSource', u'NumBldgs', u'NumFloors',
       u'UnitsRes', u'UnitsTotal', u'LotFront', u'LotDepth', u'BldgFront',
       u'BldgDepth', u'Ext', u'ProxCode', u'IrrLotCode', u'LotType',
       u'BsmtCode', u'AssessLand', u'AssessTot', u'ExemptLand', u'ExemptTot',
       u'YearBuilt', u'YearAlter1', u'YearAlter2', u'HistDist', u'Landmark',
       u'BuiltFAR', u'ResidFAR', u'CommFAR', u'

In [797]:
pluto = pluto[['NumFloors','UnitsTotal','YearBuilt','BBL']]

In [798]:
pluto = pluto[(pluto != 0).all(1)]

In [799]:
pluto.BBL = pluto.BBL.astype('str').str[:-2]

In [800]:
merged16 = pd.merge(merged16,pluto,on='BBL')

In [801]:
merged16['Age'] = 2017-merged16['YearBuilt']

In [802]:
merged16.drop([merged16.columns[-2]],axis=1,inplace=True)

In [803]:
merged16.head()

,BBL,Zip,Benchmarking_Submission,Site_EUI,Weather_Normalized_Source_EUI,Indoor_Water_Intensity(gal/ft2),Reported_Water_Method,ENERGY_STAR_Score,GHG,Property_Floor_Area(ft2),Reported_Facility_Type,Number_of_Buildings,VALUE,NumFloors,UnitsTotal,Age
0,1014270028,10021.0,In Compliance,44.9,105.2,71.51,Manual,80.0,538.4,166432.0,Multifamily Housing,1.0,203.873053,21.0,149,53
1,1015180024,10128.0,In Compliance,91.3,180.7,64.86,ABS,10.0,699.3,114939.0,Multifamily Housing,1.0,211.138082,9.0,107,57
2,1000260021,10005.0,In Compliance,63.5,128.4,34.22,ABS,39.0,2163.1,493187.0,Multifamily Housing,1.0,231.713326,27.0,440,59
3,1000520021,10006.0,In Compliance,84.6,131.1,23.11,ABS,24.0,379.0,57945.0,Multifamily Housing,1.0,105.410303,13.0,22,117
4,1000680028,10038.0,In Compliance,107.7,231.7,88.05,ABS,40.0,534.2,71539.0,Hotel,1.0,421.168873,18.0,2,19


# 2. PCA

In [804]:
X_num = merged16[list(merged16.columns[3:6])+list(merged16.columns[7:10])+['Number_of_Buildings','VALUE','NumFloors','UnitsTotal','Age']]

### Normalization

### In order to do log transformation later, here we use feature scaling normalization in case of negative numbers

In [805]:
columns = list(merged16.columns[3:6])+list(merged16.columns[7:10])+['Number_of_Buildings','VALUE','NumFloors','UnitsTotal','Age']

In [806]:
def norm(data,columns):
    for x in columns:
        data[x] = (data[x]-np.min(data[x]))/(np.max(data[x])-np.min(data[x]))
    return data

In [807]:
merged16 = norm(merged16,columns)

In [808]:
merged16.head()

,BBL,Zip,Benchmarking_Submission,Site_EUI,Weather_Normalized_Source_EUI,Indoor_Water_Intensity(gal/ft2),Reported_Water_Method,ENERGY_STAR_Score,GHG,Property_Floor_Area(ft2),Reported_Facility_Type,Number_of_Buildings,VALUE,NumFloors,UnitsTotal,Age
0,1014270028,10021.0,In Compliance,0.000692,0.000514,0.005082,Manual,0.797980,0.001965,0.086420,Multifamily Housing,0.022222,0.148629,0.40,0.116078,0.30625
1,1015180024,10128.0,In Compliance,0.001415,0.000889,0.004610,ABS,0.090909,0.002558,0.048184,Multifamily Housing,0.022222,0.154151,0.16,0.083137,0.33125
2,1000260021,10005.0,In Compliance,0.000982,0.000629,0.002432,ABS,0.383838,0.007954,0.329054,Multifamily Housing,0.022222,0.169789,0.52,0.344314,0.34375
3,1000520021,10006.0,In Compliance,0.001311,0.000643,0.001642,ABS,0.232323,0.001378,0.005862,Multifamily Housing,0.022222,0.073796,0.24,0.016471,0.70625
4,1000680028,10038.0,In Compliance,0.001671,0.001142,0.006258,ABS,0.393939,0.001950,0.015957,Hotel,0.022222,0.313778,0.34,0.000784,0.09375


### Remove outliers

In [809]:
def clean(data,columns):
    for x in columns:
        data = data[(data[x]<np.mean(data[x])+2*np.std(data[x]))&(data[x]>np.mean(data[x])-2*np.std(data[x]))]
    return data

In [810]:
merged16 = clean(merged16,columns)

In [811]:
merged16.head()

,BBL,Zip,Benchmarking_Submission,Site_EUI,Weather_Normalized_Source_EUI,Indoor_Water_Intensity(gal/ft2),Reported_Water_Method,ENERGY_STAR_Score,GHG,Property_Floor_Area(ft2),Reported_Facility_Type,Number_of_Buildings,VALUE,NumFloors,UnitsTotal,Age
1,1015180024,10128.0,In Compliance,0.001415,0.000889,0.004610,ABS,0.090909,0.002558,0.048184,Multifamily Housing,0.022222,0.154151,0.16,0.083137,0.33125
3,1000520021,10006.0,In Compliance,0.001311,0.000643,0.001642,ABS,0.232323,0.001378,0.005862,Multifamily Housing,0.022222,0.073796,0.24,0.016471,0.70625
6,1000710001,10038.0,In Compliance,0.001486,0.000743,0.002553,ABS,0.212121,0.001588,0.002168,Multifamily Housing,0.022222,0.170841,0.16,0.050980,0.51875
7,1000720027,10038.0,In Compliance,0.000550,0.000331,0.001922,ABS,0.989899,0.000429,0.001452,Multifamily Housing,0.022222,0.066798,0.14,0.018824,0.70625
10,1000920024,10038.0,In Compliance,0.000373,0.000372,0.002103,Manual,1.000000,0.000795,0.043620,Financial Office,0.022222,0.104004,0.22,0.010980,0.64375


In [812]:
X_num = merged16[list(merged16.columns[3:6])+list(merged16.columns[7:10])+['Number_of_Buildings','NumFloors','UnitsTotal','Age']]
pca = PCA(0.99)
Xproj = pca.fit_transform(X_num)

In [813]:
pca.explained_variance_ratio_

array([ 0.80296508,  0.15361489,  0.03345228])

In [814]:
pca.components_

array([[  1.11379369e-03,   4.46725175e-04,   3.18093164e-03,
         -9.99708098e-01,   1.10542838e-03,  -8.00269198e-03,
          2.37941122e-04,  -2.09437230e-02,   3.47633095e-03,
         -7.49089980e-03],
       [  8.92115697e-05,  -2.02698044e-05,   1.72308097e-04,
          9.95617200e-03,   4.73800632e-04,   2.70199125e-02,
          1.00240598e-03,  -1.17516152e-01,   8.60012704e-02,
         -9.88920253e-01],
       [ -5.16836010e-04,   2.38779878e-04,  -1.83093723e-02,
         -2.09002073e-02,   4.08915049e-03,   1.45445635e-01,
          1.09909387e-03,   9.82493333e-01,   1.92774917e-02,
         -1.11312587e-01]])

In [815]:
components_explanation = pd.DataFrame(pca.components_,columns = X_num.columns)
components_explanation

,Site_EUI,Weather_Normalized_Source_EUI,Indoor_Water_Intensity(gal/ft2),ENERGY_STAR_Score,GHG,Property_Floor_Area(ft2),Number_of_Buildings,NumFloors,UnitsTotal,Age
0,0.001114,0.000447,0.003181,-0.999708,0.001105,-0.008003,0.000238,-0.020944,0.003476,-0.007491
1,0.000089,-0.000020,0.000172,0.009956,0.000474,0.027020,0.001002,-0.117516,0.086001,-0.988920
2,-0.000517,0.000239,-0.018309,-0.020900,0.004089,0.145446,0.001099,0.982493,0.019277,-0.111313


## Here we could find the impact of each original variable on the principle components 
## Based on the coefficients, we could explain each component as:
## Component 1: Energy efficiency
## Component 2: Age
## Component 3: Stories

# 3. Analysis

In [816]:
merged16['Energy_efficiency'] = Xproj[:,0]
merged16['age'] = Xproj[:,1]
merged16['Stories'] = Xproj[:,2]

In [817]:
merged16 = merged16.drop(X_num,axis=1)

In [818]:
merged16.head()

,BBL,Zip,Benchmarking_Submission,Reported_Water_Method,Reported_Facility_Type,VALUE,Energy_efficiency,age,Stories
1,1015180024,10128.0,In Compliance,ABS,Multifamily Housing,0.154151,0.556229,0.138713,0.070187
3,1000520021,10006.0,In Compliance,ABS,Multifamily Housing,0.073796,0.410468,-0.247003,0.096698
6,1000710001,10038.0,In Compliance,ABS,Multifamily Housing,0.170841,0.433897,-0.049512,0.039504
7,1000720027,10038.0,In Compliance,ABS,Multifamily Housing,0.066798,-0.344750,-0.227627,-0.017990
10,1000920024,10038.0,In Compliance,Manual,Financial Office,0.104004,-0.356419,-0.174655,0.073336


### Hierarchical linear regression ---- to check if energy efficiency actually works

In [711]:
lm_no_energy = smf.ols(formula='VALUE~C(Benchmarking_Submission)+C(Reported_Water_Method)+\
                       C(Reported_Facility_Type)+Stories+age'
             ,data=merged16).fit()
lm_no_energy.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     192.7
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          1.49e-312
Time:                        21:29:44   Log-Likelihood:                 4556.8
No. Observations:                2214   AIC:                            -9090.
Df Residuals:                    2202   BIC:                            -9021.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                   0.0391      0.031      1.246      0.213      -0.022       0.101
C(Reported_Water_Method)[T.Manual]                          0.0116      0.005      2.403      0.016       0.002       0.021
C(Reported_Facility_Type)[T.Hotel]                          0.1180      0.044      2.672      0.008       0.031       0.205
C(Reported_Facility_Type)[T.Medical Office]                 0.0764      0.044      1.732      0.083      -0.010       0.163
C(Reported_Facility_Type)[T.Multifamily Housing]            0.0142      0.031      0.454      0.650      -0.047       0.076
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.0639      0.036      1.770      0.077      -0.007       0.135
C(Reported_Facility_Type)[T.Office]                         0.0406      0.032      1.289      0.198      -0.021       0.102
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      -0.0129      0.035     -0.368      0.713      -0.081       0.056
C(Reported_Facility_Type)[T.Retail Store]                   0.0044      0.044      0.099      0.921      -0.082       0.091
C(Reported_Facility_Type)[T.Senior Care Community]          0.1621      0.044      3.674      0.000       0.076       0.249
Stories                                                     0.5387      0.014     39.667      0.000       0.512       0.565
age                                                        -0.0788      0.006    -12.508      0.000      -0.091      -0.066
==============================================================================
Omnibus:                      489.206   Durbin-Watson:                   1.090
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1428.085
Skew:                           1.132   Prob(JB):                    7.86e-311
Kurtosis:                       6.218   Cond. No.                         204.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [713]:
lm_with_energy = smf.ols(formula='VALUE~C(Benchmarking_Submission)+C(Reported_Water_Method)+\
                       C(Reported_Facility_Type)+Stories+age+Energy_efficiency'
             ,data=merged16).fit()
lm_with_energy.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     179.8
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          1.18e-315
Time:                        21:32:06   Log-Likelihood:                 4566.7
No. Observations:                2214   AIC:                            -9107.
Df Residuals:                    2201   BIC:                            -9033.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                   0.0350      0.031      1.120      0.263      -0.026       0.096
C(Reported_Water_Method)[T.Manual]                          0.0112      0.005      2.330      0.020       0.002       0.021
C(Reported_Facility_Type)[T.Hotel]                          0.1180      0.044      2.684      0.007       0.032       0.204
C(Reported_Facility_Type)[T.Medical Office]                 0.0793      0.044      1.806      0.071      -0.007       0.165
C(Reported_Facility_Type)[T.Multifamily Housing]            0.0184      0.031      0.589      0.556      -0.043       0.080
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.0694      0.036      1.929      0.054      -0.001       0.140
C(Reported_Facility_Type)[T.Office]                         0.0431      0.031      1.373      0.170      -0.018       0.105
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      -0.0095      0.035     -0.274      0.784      -0.078       0.059
C(Reported_Facility_Type)[T.Retail Store]                   0.0056      0.044      0.127      0.899      -0.081       0.092
C(Reported_Facility_Type)[T.Senior Care Community]          0.1623      0.044      3.694      0.000       0.076       0.248
Stories                                                     0.5403      0.014     39.936      0.000       0.514       0.567
age                                                        -0.0794      0.006    -12.652      0.000      -0.092      -0.067
Energy_efficiency                                          -0.0120      0.003     -4.446      0.000      -0.017      -0.007
==============================================================================
Omnibus:                      498.490   Durbin-Watson:                   1.101
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1471.891
Skew:                           1.149   Prob(JB):                         0.00
Kurtosis:                       6.267   Cond. No.                         204.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [714]:
anova = sm.stats.anova_lm(lm_no_energy,lm_with_energy)
anova

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,2202.0,2.113400,0.0,NaN,NaN,NaN
1,2201.0,2.094591,1.0,0.018809,19.764594,0.000009


In [715]:
anova['Pr(>F)'][1] < 0.05

True

## As p here is way less than our significance level 5%, We could conclude that energy efficiency actually has a significant impact on property market value

### Linear Regression
### Optimize the model using feature selection (maximizing adj-R2 which would be increased only when adding useful regressors)

In [716]:
merged16.head()

,BBL,Zip,Benchmarking_Submission,Reported_Water_Method,Reported_Facility_Type,VALUE,Energy_efficiency,Stories,age
1,1015180024,10128.0,In Compliance,ABS,Multifamily Housing,0.154151,0.556229,0.070187,0.138713
3,1000520021,10006.0,In Compliance,ABS,Multifamily Housing,0.073796,0.410468,0.096698,-0.247003
6,1000710001,10038.0,In Compliance,ABS,Multifamily Housing,0.170841,0.433897,0.039504,-0.049512
7,1000720027,10038.0,In Compliance,ABS,Multifamily Housing,0.066798,-0.344750,-0.017990,-0.227627
10,1000920024,10038.0,In Compliance,Manual,Financial Office,0.104004,-0.356419,0.073336,-0.174655


In [748]:
train,test = train_test_split(merged16, test_size = 0.3)

In [749]:
lm1 = smf.ols(formula='VALUE~C(Benchmarking_Submission)+C(Reported_Water_Method)+\
                       C(Reported_Facility_Type)+Stories+age+Energy_efficiency'
             ,data=train).fit()
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     118.3
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          2.92e-208
Time:                        21:40:54   Log-Likelihood:                 3172.8
No. Observations:                1549   AIC:                            -6320.
Df Residuals:                    1536   BIC:                            -6250.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                   0.0343      0.032      1.072      0.284      -0.028       0.097
C(Reported_Water_Method)[T.Manual]                          0.0143      0.006      2.393      0.017       0.003       0.026
C(Reported_Facility_Type)[T.Hotel]                          0.1175      0.045      2.621      0.009       0.030       0.205
C(Reported_Facility_Type)[T.Medical Office]                 0.0816      0.045      1.825      0.068      -0.006       0.169
C(Reported_Facility_Type)[T.Multifamily Housing]            0.0190      0.032      0.596      0.551      -0.044       0.082
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.0707      0.037      1.925      0.054      -0.001       0.143
C(Reported_Facility_Type)[T.Office]                         0.0449      0.032      1.397      0.162      -0.018       0.108
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      -0.0278      0.039     -0.715      0.475      -0.104       0.048
C(Reported_Facility_Type)[T.Retail Store]                   0.0075      0.045      0.167      0.867      -0.080       0.095
C(Reported_Facility_Type)[T.Senior Care Community]          0.1627      0.045      3.634      0.000       0.075       0.251
Stories                                                     0.5277      0.016     32.458      0.000       0.496       0.560
age                                                        -0.0724      0.008     -9.638      0.000      -0.087      -0.058
Energy_efficiency                                          -0.0113      0.003     -3.408      0.001      -0.018      -0.005
==============================================================================
Omnibus:                      414.211   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1281.101
Skew:                           1.334   Prob(JB):                    6.49e-279
Kurtosis:                       6.568   Cond. No.                         172.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [750]:
pvalues = lm1.pvalues

In [751]:
pvalues.sort_values(ascending=True)

Stories                                                    2.000406e-176
age                                                         2.187010e-21
C(Reported_Facility_Type)[T.Senior Care Community]          2.883538e-04
Energy_efficiency                                           6.724888e-04
C(Reported_Facility_Type)[T.Hotel]                          8.858720e-03
C(Reported_Water_Method)[T.Manual]                          1.683174e-02
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     5.440749e-02
C(Reported_Facility_Type)[T.Medical Office]                 6.824039e-02
C(Reported_Facility_Type)[T.Office]                         1.624775e-01
Intercept                                                   2.837653e-01
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]       4.749804e-01
C(Reported_Facility_Type)[T.Multifamily Housing]            5.514878e-01
C(Reported_Facility_Type)[T.Retail Store]                   8.673970e-01
dtype: float64

In [752]:
def AdjR2(features):
    lm = smf.ols(formula = ('VALUE~'+features), data = train).fit()
    lmy = lm.predict(test)
    y_err = lmy-test.VALUE
    y_norm = test.VALUE-np.mean(test.VALUE)
    # Adjusted R^2
    R2 = 1 - y_err.dot(y_err) / y_norm.dot(y_norm) * (len(merged16.index)-1) / (len(merged16.index)-(features.count('+')+1+features.count('-'))-1)
    return R2, lm

In [753]:
AdjR2('Stories-1')[0]

-1.1121744957088757

In [754]:
AdjR2('Stories-1+age')[0]

-1.0630039272190839

In [755]:
AdjR2('Stories-1+age+Energy_efficiency')[0]

-1.0698769648531985

In [756]:
AdjR2('Stories-1+age+Energy_efficiency+C(Reported_Water_Method)')[0]

0.52044003798713212

In [757]:
AdjR2('Stories-1+age+Energy_efficiency+C(Reported_Water_Method)+C(Reported_Facility_Type)')[0]

0.52457320151373055

In [758]:
AdjR2('Stories+age+Energy_efficiency+C(Reported_Water_Method)+C(Reported_Facility_Type)')[0]

0.5247885216217405

In [761]:
lm = smf.ols(formula='VALUE~Stories+age+Energy_efficiency+C(Reported_Water_Method)+C(Reported_Facility_Type)'
             ,data=train).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     118.3
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          2.92e-208
Time:                        21:45:07   Log-Likelihood:                 3172.8
No. Observations:                1549   AIC:                            -6320.
Df Residuals:                    1536   BIC:                            -6250.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                   0.0343      0.032      1.072      0.284      -0.028       0.097
C(Reported_Water_Method)[T.Manual]                          0.0143      0.006      2.393      0.017       0.003       0.026
C(Reported_Facility_Type)[T.Hotel]                          0.1175      0.045      2.621      0.009       0.030       0.205
C(Reported_Facility_Type)[T.Medical Office]                 0.0816      0.045      1.825      0.068      -0.006       0.169
C(Reported_Facility_Type)[T.Multifamily Housing]            0.0190      0.032      0.596      0.551      -0.044       0.082
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.0707      0.037      1.925      0.054      -0.001       0.143
C(Reported_Facility_Type)[T.Office]                         0.0449      0.032      1.397      0.162      -0.018       0.108
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      -0.0278      0.039     -0.715      0.475      -0.104       0.048
C(Reported_Facility_Type)[T.Retail Store]                   0.0075      0.045      0.167      0.867      -0.080       0.095
C(Reported_Facility_Type)[T.Senior Care Community]          0.1627      0.045      3.634      0.000       0.075       0.251
Stories                                                     0.5277      0.016     32.458      0.000       0.496       0.560
age                                                        -0.0724      0.008     -9.638      0.000      -0.087      -0.058
Energy_efficiency                                          -0.0113      0.003     -3.408      0.001      -0.018      -0.005
==============================================================================
Omnibus:                      414.211   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1281.101
Skew:                           1.334   Prob(JB):                    6.49e-279
Kurtosis:                       6.568   Cond. No.                         172.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Log-log Linear Regression

### Logarithm of both x and y

In [766]:
merged16.head()

,BBL,Zip,Benchmarking_Submission,Reported_Water_Method,Reported_Facility_Type,VALUE,Energy_efficiency,Stories,age
1,1015180024,10128.0,In Compliance,ABS,Multifamily Housing,0.154151,0.556229,0.070187,0.138713
3,1000520021,10006.0,In Compliance,ABS,Multifamily Housing,0.073796,0.410468,0.096698,-0.247003
6,1000710001,10038.0,In Compliance,ABS,Multifamily Housing,0.170841,0.433897,0.039504,-0.049512
7,1000720027,10038.0,In Compliance,ABS,Multifamily Housing,0.066798,-0.344750,-0.017990,-0.227627
10,1000920024,10038.0,In Compliance,Manual,Financial Office,0.104004,-0.356419,0.073336,-0.174655


In [820]:
columns = ['VALUE','Energy_efficiency','Stories','age']

### As here energy_efficiency has negative numbers, we would add 1 to each of them

In [821]:
loglog = merged16.copy()
loglog['Energy_efficiency'] = loglog['Energy_efficiency']+1
loglog['Stories'] = loglog['Stories']+1
loglog['age'] = loglog['age']+1

In [823]:
for column in columns:
    loglog[column] = np.log(loglog[column])

C:\Users\sddz0\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


### remove inf and 0

In [827]:
loglog = loglog.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [828]:
train, test = train_test_split(loglog,test_size = 0.3)

In [829]:
loglog1 = smf.ols(formula='VALUE~C(Benchmarking_Submission)+C(Reported_Water_Method)+\
                       C(Reported_Facility_Type)+Stories+age+Energy_efficiency'
             ,data=train).fit()
loglog1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     77.79
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          7.62e-149
Time:                        21:59:27   Log-Likelihood:                -1347.8
No. Observations:                1549   AIC:                             2722.
Df Residuals:                    1536   BIC:                             2791.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                  -3.1880      0.590     -5.401      0.000      -4.346      -2.030
C(Reported_Water_Method)[T.Manual]                          0.1452      0.101      1.437      0.151      -0.053       0.343
C(Reported_Facility_Type)[T.Hotel]                          1.4403      0.828      1.739      0.082      -0.185       3.065
C(Reported_Facility_Type)[T.Medical Office]                 0.6505      0.827      0.787      0.432      -0.972       2.273
C(Reported_Facility_Type)[T.Multifamily Housing]           -0.0035      0.590     -0.006      0.995      -1.161       1.154
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.6800      0.719      0.946      0.344      -0.730       2.090
C(Reported_Facility_Type)[T.Office]                         0.4502      0.593      0.759      0.448      -0.713       1.614
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      -0.1051      0.657     -0.160      0.873      -1.393       1.183
C(Reported_Facility_Type)[T.Retail Store]                   0.1433      0.827      0.173      0.862      -1.478       1.765
C(Reported_Facility_Type)[T.Senior Care Community]          1.8127      0.828      2.190      0.029       0.189       3.436
Stories                                                     8.4396      0.323     26.119      0.000       7.806       9.073
age                                                        -0.6060      0.143     -4.244      0.000      -0.886      -0.326
Energy_efficiency                                          -0.1496      0.062     -2.405      0.016      -0.272      -0.028
==============================================================================
Omnibus:                       15.607   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.042
Skew:                          -0.073   Prob(JB):                     9.92e-06
Kurtosis:                       3.579   Cond. No.                         171.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [830]:
pvalues = loglog1.pvalues

In [831]:
pvalues.sort_values(ascending=True)

Stories                                                    9.599460e-125
Intercept                                                   7.663941e-08
age                                                         2.330749e-05
Energy_efficiency                                           1.629812e-02
C(Reported_Facility_Type)[T.Senior Care Community]          2.864804e-02
C(Reported_Facility_Type)[T.Hotel]                          8.232303e-02
C(Reported_Water_Method)[T.Manual]                          1.510207e-01
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     3.442486e-01
C(Reported_Facility_Type)[T.Medical Office]                 4.316470e-01
C(Reported_Facility_Type)[T.Office]                         4.480535e-01
C(Reported_Facility_Type)[T.Retail Store]                   8.623642e-01
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]       8.728667e-01
C(Reported_Facility_Type)[T.Multifamily Housing]            9.953200e-01
dtype: float64

In [833]:
AdjR2('Stories-1')[0]

-18.871374706242285

In [834]:
AdjR2('Stories')[0]

0.38658611251981623

In [835]:
AdjR2('Stories+age')[0]

0.40423371297921129

In [836]:
AdjR2('Stories+age+Energy_efficiency')[0]

0.41403306701536913

In [837]:
AdjR2('Stories+age+Energy_efficiency+C(Reported_Facility_Type)')[0]

0.42045735431246034

In [838]:
AdjR2('Stories+age+Energy_efficiency+C(Reported_Facility_Type)+C(Reported_Water_Method)')[0]

0.42078641649131432

In [845]:
semilog1 = smf.ols(formula='VALUE~C(Benchmarking_Submission)+C(Reported_Water_Method)+\
                       C(Reported_Facility_Type)+Stories+age+Energy_efficiency'
             ,data=train).fit()
semilog1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.385
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     96.34
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          1.19e-154
Time:                        22:03:19   Log-Likelihood:                -1335.7
No. Observations:                1549   AIC:                             2693.
Df Residuals:                    1538   BIC:                             2752.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                  -1.7535      0.576     -3.044      0.002      -2.883      -0.624
C(Reported_Water_Method)[T.Manual]                          0.1768      0.105      1.687      0.092      -0.029       0.382
C(Reported_Facility_Type)[T.Medical Office]                -0.7986      0.815     -0.980      0.327      -2.397       0.800
C(Reported_Facility_Type)[T.Multifamily Housing]           -1.4491      0.576     -2.515      0.012      -2.579      -0.319
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]    -0.3715      0.817     -0.455      0.649      -1.974       1.231
C(Reported_Facility_Type)[T.Office]                        -0.9999      0.582     -1.718      0.086      -2.141       0.142
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      -1.5222      0.645     -2.359      0.018      -2.788      -0.256
C(Reported_Facility_Type)[T.Senior Care Community]          0.3606      0.814      0.443      0.658      -1.236       1.957
Stories                                                     8.2837      0.309     26.793      0.000       7.677       8.890
age                                                        -0.5867      0.143     -4.106      0.000      -0.867      -0.306
Energy_efficiency                                          -0.1792      0.060     -3.012      0.003      -0.296      -0.062
==============================================================================
Omnibus:                        7.310   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.026   Jarque-Bera (JB):                9.353
Skew:                          -0.020   Prob(JB):                      0.00931
Kurtosis:                       3.378   Cond. No.                         150.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Semi-log Linear Regression

In [840]:
semilog = merged16.copy()

In [841]:
semilog.VALUE = np.log(semilog.VALUE)

C:\Users\sddz0\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [842]:
semilog = semilog.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [882]:
train, test = train_test_split(semilog,test_size = 0.3)

In [895]:
semilog1 = smf.ols(formula='VALUE~C(Benchmarking_Submission)+C(Reported_Water_Method)+\
                       C(Reported_Facility_Type)+Stories+age+Energy_efficiency'
             ,data=train).fit()
semilog1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.380
Method:                 Least Squares   F-statistic:                     79.96
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          2.59e-152
Time:                        22:08:19   Log-Likelihood:                -1352.7
No. Observations:                1549   AIC:                             2731.
Df Residuals:                    1536   BIC:                             2801.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                  -3.1120      0.592     -5.255      0.000      -4.274      -1.950
C(Reported_Water_Method)[T.Manual]                          0.0807      0.103      0.786      0.432      -0.121       0.282
C(Reported_Facility_Type)[T.Hotel]                          1.3478      0.831      1.621      0.105      -0.283       2.978
C(Reported_Facility_Type)[T.Medical Office]                 0.5770      0.830      0.695      0.487      -1.050       2.204
C(Reported_Facility_Type)[T.Multifamily Housing]           -0.0926      0.592     -0.156      0.876      -1.254       1.069
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.6487      0.721      0.900      0.368      -0.766       2.063
C(Reported_Facility_Type)[T.Office]                         0.3611      0.595      0.607      0.544      -0.806       1.529
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      -0.0170      0.680     -0.025      0.980      -1.351       1.317
C(Reported_Facility_Type)[T.Retail Store]                   0.0700      0.829      0.084      0.933      -1.557       1.697
C(Reported_Facility_Type)[T.Senior Care Community]          1.6998      0.830      2.047      0.041       0.071       3.328
Stories                                                     8.0395      0.301     26.682      0.000       7.448       8.630
age                                                        -0.6170      0.143     -4.308      0.000      -0.898      -0.336
Energy_efficiency                                          -0.1981      0.061     -3.240      0.001      -0.318      -0.078
==============================================================================
Omnibus:                       12.312   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               16.516
Skew:                          -0.084   Prob(JB):                     0.000259
Kurtosis:                       3.477   Cond. No.                         171.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [884]:
pvalues = semilog1.pvalues

In [885]:
pvalues.sort_values(ascending=True)

Intercept                                                  0.000367
Energy_efficiency                                          0.029035
C(Reported_Water_Method)[T.Manual]                         0.057119
C(Reported_Facility_Type)[T.Multifamily Housing]           0.346200
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]    0.432367
C(Reported_Facility_Type)[T.Senior Care Community]         0.446481
C(Reported_Facility_Type)[T.Medical Office]                0.453108
C(Reported_Facility_Type)[T.Hotel]                         0.665501
C(Reported_Facility_Type)[T.Office]                        0.668861
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      0.800121
C(Reported_Facility_Type)[T.Retail Store]                  0.810781
dtype: float64

In [886]:
AdjR2('Stories-1')[0]

-19.492403034402301

In [887]:
AdjR2('Stories-1+age')[0]

-19.508398731969162

In [888]:
AdjR2('Stories+age')[0]

0.37375280072376516

In [889]:
AdjR2('Stories+age+Energy_efficiency')[0]

0.38123638527862769

In [891]:
AdjR2('Stories+age+Energy_efficiency+C(Reported_Water_Method)')[0]

0.38214599618876299

In [892]:
AdjR2('Stories+age+Energy_efficiency+C(Reported_Water_Method)+C(Reported_Facility_Type)')[0]

0.39204334346689085

In [893]:
lm = smf.ols(formula='VALUE~Stories+age+Energy_efficiency+C(Reported_Water_Method)+C(Reported_Facility_Type)'
             ,data=train).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.380
Method:                 Least Squares   F-statistic:                     79.96
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          2.59e-152
Time:                        22:07:15   Log-Likelihood:                -1352.7
No. Observations:                1549   AIC:                             2731.
Df Residuals:                    1536   BIC:                             2801.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                  -3.1120      0.592     -5.255      0.000      -4.274      -1.950
C(Reported_Water_Method)[T.Manual]                          0.0807      0.103      0.786      0.432      -0.121       0.282
C(Reported_Facility_Type)[T.Hotel]                          1.3478      0.831      1.621      0.105      -0.283       2.978
C(Reported_Facility_Type)[T.Medical Office]                 0.5770      0.830      0.695      0.487      -1.050       2.204
C(Reported_Facility_Type)[T.Multifamily Housing]           -0.0926      0.592     -0.156      0.876      -1.254       1.069
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.6487      0.721      0.900      0.368      -0.766       2.063
C(Reported_Facility_Type)[T.Office]                         0.3611      0.595      0.607      0.544      -0.806       1.529
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      -0.0170      0.680     -0.025      0.980      -1.351       1.317
C(Reported_Facility_Type)[T.Retail Store]                   0.0700      0.829      0.084      0.933      -1.557       1.697
C(Reported_Facility_Type)[T.Senior Care Community]          1.6998      0.830      2.047      0.041       0.071       3.328
Stories                                                     8.0395      0.301     26.682      0.000       7.448       8.630
age                                                        -0.6170      0.143     -4.308      0.000      -0.898      -0.336
Energy_efficiency                                          -0.1981      0.061     -3.240      0.001      -0.318      -0.078
==============================================================================
Omnibus:                       12.312   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               16.516
Skew:                          -0.084   Prob(JB):                     0.000259
Kurtosis:                       3.477   Cond. No.                         171.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""